In [ ]:
from wc_simd.utility import spark_path

In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    model_kwargs=dict(temperature=0),
    # other params...
)

In [2]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("test_pyspark") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()
spark

25/04/16 15:56:26 WARN Utils: Your hostname, WTH2X9T5QYWX resolves to a loopback address: 127.0.0.1; using 10.21.24.66 instead (on interface en0)
25/04/16 15:56:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 15:56:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df_works = spark.read.parquet(spark_path("../data/imports/works.parquet"))

In [6]:
works_schema = df_works.schema
works_schema

StructType([StructField('alternativeTitles', ArrayType(StringType(), True), True), StructField('availabilities', ArrayType(StructType([StructField('id', StringType(), True), StructField('label', StringType(), True), StructField('type', StringType(), True)]), True), True), StructField('contributors', ArrayType(StructType([StructField('agent', StructType([StructField('id', StringType(), True), StructField('identifiers', ArrayType(StructType([StructField('identifierType', StructType([StructField('id', StringType(), True), StructField('label', StringType(), True), StructField('type', StringType(), True)]), True), StructField('type', StringType(), True), StructField('value', StringType(), True)]), True), True), StructField('label', StringType(), True), StructField('type', StringType(), True)]), True), StructField('primary', BooleanType(), True), StructField('roles', ArrayType(StructType([StructField('label', StringType(), True), StructField('type', StringType(), True)]), True), True), Struc

In [1]:
# load data/works.dbml text file
with open("../data/works.dbml", "r") as f:
    works_dbml = f.read()
works_dbml

'/* Main record table */\nTable Records {\n  id varchar [pk]                 // from main "id"\n  currentFrequency varchar\n  description text\n  duration bigint\n  edition varchar\n  physicalDescription text\n  referenceNumber varchar\n  title varchar\n  type varchar\n}\n\n/* alternativeTitles is an array of strings */\nTable AlternativeTitles {\n  id int [pk, increment]\n  record_id varchar [ref: > Records.id]\n  title varchar\n}\n\n/* availabilities is an array of structs */\nTable Availabilities {\n  id int [pk, increment]\n  record_id varchar [ref: > Records.id]\n  availability_id varchar         // from availabilities.id\n  label varchar                   // from availabilities.label\n  type varchar                    // from availabilities.type\n}\n\n/* contributors is an array of structs. \n   Here we break out the contributor’s core info. */\nTable Contributors {\n  id int [pk, increment]\n  record_id varchar [ref: > Records.id]\n  contributor_type varchar        // from contr

## Bootstrap Prompt for SQL Generation

In [ ]:
# table_name = "GenreConceptIdentifiers"
# table_name = "ContributorAgentIdentifiers"
table_name = "HoldingLocationLocationTypes"
# table_name = "ItemLocationAccessConditions"

messages = [
    ("system",
     "You are a helpful assistant that generates schema documentation and/or a data dictionary for a Spark Dataframe",),
    ("human", f"""I will give you a spark dataframe schema and a dbml file.
SCHEMA:
{works_schema}

DBML:
{works_dbml}

Given a name of a nested table, I want you to generate the spark SQL statement to show the contents of it.

EXAMPLE 1:
Table: ProductionAgents

SELECT col.* FROM (
  SELECT EXPLODE(agents) as col FROM (
    SELECT col.* FROM (
      SELECT EXPLODE(production) as col FROM works
    )
  )
)

EXAMPLE 2:
Table: ContributorAgentIdentifiers

SELECT col.* FROM (
  (SELECT EXPLODE(identifiers) as col
    FROM (
      SELECT agent.*
      FROM (
        SELECT col.*
        FROM (
          SELECT EXPLODE(contributors) as col
          FROM works
        )
      )
    )
  )
)

Now give me the spark SQL statement for the table {table_name} in the schema above. Just the sql statement, nothing else.
"""
    ),]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

SELECT col.* FROM (
  SELECT locationType.*
  FROM (
    SELECT location.*
    FROM (
      SELECT EXPLODE(holdings) as col
      FROM works
    )
  )
)


## Test Generated SQL

In [53]:
spark.sql("""
SELECT col.* FROM (
  SELECT EXPLODE(accessConditions) as col
  FROM (
    SELECT locations.*
    FROM (
      SELECT EXPLODE(locations) as locations
      FROM (
        SELECT col.*
        FROM (
          SELECT EXPLODE(items) as col
          FROM works
        )
      )
    )
  )
)
""").show(truncate=False)

+----------------------------------------------+----+------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|method                                        |note|status                                                |terms                                                                                                                                                                                                                                                                                                                        |type           |
+----------------------------------------------+----+------------------------------------------------------+------

In [42]:
spark.sql("""
SELECT col.* FROM (
  SELECT EXPLODE(col.roles) as col
  FROM (
    SELECT EXPLODE(contributors) as col
    FROM works
  )
)
""").show(truncate=False)

+----------------+----------------+
|label           |type            |
+----------------+----------------+
|data contributor|ContributionRole|
|architect       |ContributionRole|
|data contributor|ContributionRole|
|data contributor|ContributionRole|
|printer         |ContributionRole|
|former owner    |ContributionRole|
|data contributor|ContributionRole|
|data contributor|ContributionRole|
|author,         |ContributionRole|
|artist          |ContributionRole|
|author          |ContributionRole|
|author          |ContributionRole|
|author          |ContributionRole|
|author          |ContributionRole|
|author,         |ContributionRole|
|artist          |ContributionRole|
|author          |ContributionRole|
|data contributor|ContributionRole|
|data contributor|ContributionRole|
|data contributor|ContributionRole|
+----------------+----------------+
only showing top 20 rows



In [41]:
spark.sql("""
    SELECT EXPLODE(contributors) as col
    FROM works
""").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                                                                                                 |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{{r88uf8gp, [{{label-derived, Identifier derived from the label of the referent, IdentifierType}, Identifier, berni, francesco}], Berni, Francesco, Person}, true, [], Contributor}                                                 |
|{{guzs2skt, [{{label-derived, Identifier derived from the label of the refe

In [ ]:
spark.sql("""
SELECT locationType.* FROM (
  SELECT location.locationType
  FROM (
    SELECT col.location
    FROM (
      SELECT EXPLODE(holdings) as col
      FROM works
    )
  )
)
""").show(truncate=False)

+---------------+---------------+------------+
|id             |label          |type        |
+---------------+---------------+------------+
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|closed-stores  |Closed stores  |LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|open-shelves   |Open shelves   |LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resource|Online resource|LocationType|
|online-resou

## Generate SQL for each DBML table

In [ ]:
from pydbml import PyDBML
from pathlib import Path
parsed = PyDBML(Path('../data/works.dbml'))

In [38]:
import os


print("Number of tables:", len(parsed.tables))
for table in parsed.tables:
    print("------")
    print(table.name)
    file_path = f"out/nested_tables_queries/{table.name}.sql"
    if os.path.exists(file_path):
        print(f"Table {table.name} already exists. Skipping...")
        continue
  
    messages = [
        ("system",
         "You are a helpful assistant that generates schema documentation and/or a data dictionary for a Spark Dataframe",
         ),
        ("human",
         f"""I will give you a spark dataframe schema and a DBML file which has been derived from the schema.
DATAFRAME SCHEMA:
{works_schema}

DBML:
{works_dbml}

INSTRUCTIONS:
- Given a name of a nested table, I want you to generate the spark SQL statement to show the contents of it.
- The SQL statement should be formatted to be run in PySpark SQL.
- If table has column which are arrays or structs, exclude those columns in the result.
- Name the resulting columns according to the DATAFRAME SCHEMA column names.


EXAMPLE 1:
Table: ProductionAgents

SELECT col.* FROM (
  SELECT EXPLODE(agents) as col FROM (
    SELECT col.* FROM (
      SELECT EXPLODE(production) as col FROM works
    )
  )
)


EXAMPLE 2:
Table: ContributorAgentIdentifiers

SELECT col.* FROM (
  (SELECT EXPLODE(identifiers) as col
    FROM (
      SELECT agent.*
      FROM (
        SELECT col.*
        FROM (
          SELECT EXPLODE(contributors) as col
          FROM works
        )
      )
    )
  )
)


EXAMPLE 3:
Table: Contributors

SELECT col.primary AS primary, col.type AS type FROM (
  SELECT EXPLODE(contributors) as col
  FROM works
)


EXAMPLE 4:
Table: ContributorAgents

SELECT col.agent.id AS id, col.agent.label AS label, col.agent.type AS type FROM (
  SELECT EXPLODE(contributors) as col
  FROM works
)

Now give me the spark SQL statement for the table {table.name} in the schema above. Just the sql statement, nothing else.
"""),
        ]
    ai_msg = llm.invoke(messages)
    with open(file_path, "w") as f:
        f.write(ai_msg.content)
    print(f"Saved SQL query for {table.name} to {file_path}")

Number of tables: 47
------
Records
Saved SQL query for Records to out/nested_tables_queries/Records.sql
------
AlternativeTitles
Saved SQL query for AlternativeTitles to out/nested_tables_queries/AlternativeTitles.sql
------
Availabilities
Saved SQL query for Availabilities to out/nested_tables_queries/Availabilities.sql
------
Contributors
Saved SQL query for Contributors to out/nested_tables_queries/Contributors.sql
------
ContributorAgents
Saved SQL query for ContributorAgents to out/nested_tables_queries/ContributorAgents.sql
------
ContributorAgentIdentifiers
Saved SQL query for ContributorAgentIdentifiers to out/nested_tables_queries/ContributorAgentIdentifiers.sql
------
ContributorRoles
Saved SQL query for ContributorRoles to out/nested_tables_queries/ContributorRoles.sql
------
CreatedDate
Saved SQL query for CreatedDate to out/nested_tables_queries/CreatedDate.sql
------
Designations
Saved SQL query for Designations to out/nested_tables_queries/Designations.sql
------
Former

In [43]:
print("Number of tables:", len(parsed.tables))
for table in parsed.tables:
    print("------")
    print(table.name)
    csv_file_path = f"out/nested_tables_csv/{table.name}.csv"
    if os.path.exists(csv_file_path):
        print(f"Table {table.name}.csv already exists. Skipping...")
        continue

    sql_file_path = f"out/nested_tables_queries/{table.name}.sql"
    with open(sql_file_path, "r") as f:
        sql_query = f.read()
    df = spark.sql(sql_query)
    pd_df = df.toPandas()
    pd_df.to_csv(csv_file_path, index=False)
    print(f"Saved CSV for {table.name} to {csv_file_path}")

Number of tables: 47
------
Records
Table Records.csv already exists. Skipping...
------
AlternativeTitles
Table AlternativeTitles.csv already exists. Skipping...
------
Availabilities
Table Availabilities.csv already exists. Skipping...
------
Contributors
Table Contributors.csv already exists. Skipping...
------
ContributorAgents
Table ContributorAgents.csv already exists. Skipping...
------
ContributorAgentIdentifiers
Table ContributorAgentIdentifiers.csv already exists. Skipping...
------
ContributorRoles
Saved CSV for ContributorRoles to out/nested_tables_csv/ContributorRoles.csv
------
CreatedDate
Saved CSV for CreatedDate to out/nested_tables_csv/CreatedDate.csv
------
Designations
Saved CSV for Designations to out/nested_tables_csv/Designations.csv
------
FormerFrequencies
Saved CSV for FormerFrequencies to out/nested_tables_csv/FormerFrequencies.csv
------
Genres
Saved CSV for Genres to out/nested_tables_csv/Genres.csv
------
GenreConcepts
Saved CSV for GenreConcepts to out/ne

Saved CSV for Holdings to out/nested_tables_csv/Holdings.csv
------
HoldingEnumerations


AnalysisException: Can only star expand struct data types. Attribute: `ArrayBuffer(col)`.; line 1 pos 7